Extract RSEM counts from normal sample and merge into a dataframe  (downloaded from ENCODE)

In [ ]:
import boto3
import pandas as pd

# AWS cilents
s3_client = boto3.client('s3')

# Find filename of all cancer tsv files for input into DESeq
print('Getting df of normal sample')

# Find filename of all normal tsv files for input into DESeq
print('Getting df of normal counts')
normal_data = []
for obj in s3_client.list_objects(Bucket='zb4171', Prefix='kazoo/data_array/')['Contents']:
        obj_key = obj['Key']
        if (obj_key.endswith('.tsv')) and (obj_key.split('/')[-1].startswith('ENCFF')):
            sample_id = obj_key.split('/')[-1].split('.')[-2]
            tsv_file = s3_client.get_object(Bucket='zb4171', Key=obj_key)['Body']
            df = pd.read_csv(tsv_file, sep='\t')
            df = df[['gene_id', 'expected_count']].rename(columns={'expected_count': sample_id})
            normal_data.append(df)
            
# Make count matrix
print('Make count matrix')
df_counts = pd.DataFrame()
for df in normal_data:
    if df_counts.empty:
        df_counts = df
    else:
        df_counts = df_counts.merge(df, on='gene_id', how='inner')


# Fill na with 0 and remove rows with all 0
df_counts = df_counts.fillna(0)

df_counts.to_csv('normal_counts.tsv', sep='\t')

Convert Ensembl ID to gene symbol

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('normal_counts.tsv',delimiter='\t')
df.drop(columns='Unnamed: 0', inplace=True)

Initialise the mapper dictionary 

In [ ]:
dfmapper = pd.read_csv('mapper.csv')
mapperDict = {}
for ind, row in dfmapper.iterrows():
    if row[0] not in mapperDict:
        mapperDict[row[0]] = row[1]
    else:
        print(row)

In [ ]:
newGeneID= []
for ind, row in df.iterrows():
    if row[0] in mapperDict:
        newGeneID.append(mapperDict[row[0]])
    else:
        newGeneID.append(np.nan) 

In [ ]:
df['gene_id'] = newGeneID

Remove any Ensembl id that does not have a gene symbol

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.to_csv('normal_counts.csv',index=False)